In [2]:
from google.colab import drive, auth
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!apt-get -y update && apt-get -y install colmap
!pip install --quiet pillow piexif pillow-heif
!git clone --quiet https://github.com/NVlabs/instant-ngp
%cd instant-ngp/scripts
!curl -s https://raw.githubusercontent.com/NVlabs/instant-ngp/master/scripts/colmap2nerf.py -o colmap2nerf.py
%cd /content

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,683 kB]
Get:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,726 kB]
Get:13 http://archive.ubuntu.com/ubunt

In [ ]:
# directory setup
from pathlib import Path

DATA_ROOT = Path('/content/drive/My Drive/cs231n_nerf/datasets')
RAW_ROOT = DATA_ROOT / 'iphone_raw'

PROCESSED_ROOT = Path('/content/drive/My Drive/cs231n_nerf/datasets/iphone_processed')

ORIG_ROOT = PROCESSED_ROOT / 'original_processed'
ALLSHOT_ROOT = PROCESSED_ROOT / 'allshot_hashnerf_processed'
FEWSHOT_ROOT = PROCESSED_ROOT / 'fewshot_hashnerf_processed'

for root in (PROCESSED_ROOT, ORIG_ROOT, ALLSHOT_ROOT, FEWSHOT_ROOT):
  root.mkdir(parents=True, exist_ok=True)

In [ ]:
# processing of one specified room folder in iphone_raw/

import numpy as np
import os, subprocess, json, glob, piexif, pillow_heif, math, shutil
from pathlib import Path
from PIL import Image, ImageOps

# CHANGE NAME OF FOLDER FOR YOUR USE
room_name = 'suites_bedroom'
TARGET_SIDE = 1600 # resize long side of image for memory constraints

room_dir = RAW_ROOT / room_name
if not room_dir.is_dir():
  raise FileNotFoundError(f"Room folder {room_dir} could not be found")

# exposure value (EV) calculation for varying iso
def shutter_iso_to_ev(shutter, iso):
  return math.log2((100 * shutter) / iso)

def estimate_ev_linear(img_path, L0=0.18, pct=(5, 95)):
  rgb = np.array(Image.open(img_path), dtype=np.float32) / 255.0
  mask = rgb <= 0.04045
  lin = np.where(mask, rgb/12.92, ((rgb+0.055)/1.055)**2.4)

  Y = 0.2126*lin[...,0] + 0.7152*lin[...,1] + 0.0722*lin[...,2]
  low, high = np.percentile(Y, pct)
  Y_trim = Y[(Y >= low) & (Y <= high)]
  L = float(Y_trim.mean())
  return np.log2(L / L0)

# invoke shell
def run(cmd):
  subprocess.run(cmd, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

# scratch directories
work_dir = Path(f'/content/tmp_{room_name}')
imgs_out = work_dir / 'images'
sparse = work_dir / 'sparse'
db_path = work_dir / 'colmap.db'

shutil.rmtree(work_dir, ignore_errors=True)
imgs_out.mkdir(parents=True, exist_ok=True)

all_srcs = sorted([p for p in room_dir.iterdir() if p.suffix.lower() in {'.jpg', '.png', '.heic'}])

# precompute EVs in same order
ev_list = []
for src in all_srcs:
  exif_dict = piexif.load(str(src))
  exif_ifd = exif_dict.get('Exif', {})

  # raw_shutter = exif_ifd.get(piexif.ExifIFD.ExposureTime, None)
  # shutter = (raw_shutter[0] / raw_shutter[1]) if isinstance(raw_shutter, tuple) else (1 / 60)
  # raw_iso = exif_ifd.get(piexif.ExifIFD.ISOSpeedRatings, None)
  # iso = raw_iso if isinstance(raw_iso, int) else 100
  # ev_list.append(shutter_iso_to_ev(shutter, iso))

  raw_shutter = exif_ifd.get(piexif.ExifIFD.ExposureTime, None)
  raw_iso = exif_ifd.get(piexif.ExifIFD.ISOSpeedRatings, None)
  if raw_shutter and isinstance(raw_shutter, tuple) and isinstance(raw_iso, int):
    shutter = raw_shutter[0] / raw_shutter[1]
    iso = raw_iso
    ev = shutter_iso_to_ev(shutter, iso)
  else:
    ev = estimate_ev_linear(src)
  ev_list.append(ev)


# copy + rotate + scale down, saving as PNG
for i, src in enumerate(all_srcs):
  img = Image.open(src)
  # ONLY ROTATE FOR THESE TYPE OF IPHONE IMAGES
  img = img.rotate(-90, expand=True)
  img.thumbnail((TARGET_SIDE, TARGET_SIDE), Image.LANCZOS)
  img.save(imgs_out/f'{i:08d}.png', format='PNG', quality=95)

# feature extraction
run(f"colmap feature_extractor "
    f"--database_path {db_path} "
    f"--image_path {imgs_out} "
    f"--ImageReader.single_camera 1 "
    f"--ImageReader.camera_model SIMPLE_PINHOLE "
    f"--ImageReader.default_focal_length_factor 1 "
    f"--SiftExtraction.use_gpu 0 "
    f"--SiftExtraction.max_image_size {TARGET_SIDE}")

# sequential (OR can choose exhaustive) matching
run(f"colmap sequential_matcher "
    f"--database_path {db_path} "
    f"--SiftMatching.use_gpu 0 "
    f"--SiftMatching.guided_matching 1")

# mapping (structure -> motion)
shutil.rmtree(sparse, ignore_errors=True)
sparse.mkdir(parents=True, exist_ok=True)

run(f"colmap mapper "
    f"--database_path {db_path} "
    f"--image_path {imgs_out} "
    f"--output_path {sparse} "
    f"--Mapper.num_threads 8 "
    f"--Mapper.init_min_tri_angle 4")

# get largest model
model_dir = max(sparse.glob('*'), key=lambda p: (p / 'images.bin').stat().st_size)
sparse_txt = work_dir / 'sparse_text'
sparse_txt.mkdir(parents=True, exist_ok=True)
run(f"colmap model_converter "
    f"--input_path {model_dir} "
    f"--output_path {sparse_txt} "
    f"--output_type TXT")

# COLMAP to transforms.json (Instant-NGP compatible)
%cd /content/instant-ngp/scripts
transforms = work_dir / 'transforms.json'
run(f"python colmap2nerf.py "
    f"--colmap_matcher sequential "
    f"--images {imgs_out} "
    f"--text {sparse_txt} "
    f"--out {transforms}")
%cd /content

# attach precomputed EVs
data = json.load(open(transforms))
for frame in data['frames']:
  idx = int(Path(frame['file_path']).stem)
  frame['exposure_val'] = ev_list[idx]
json.dump(data, open(work_dir / 'transforms.json', 'w'), indent=2)

# get this processed work_dir into Drive (permanent)
final_dir = ORIG_ROOT / room_name
shutil.rmtree(final_dir, ignore_errors=True) # OVERWRITE IF RERUN
shutil.move(str(work_dir), final_dir)
print(f'Done with processing {room_name} to {final_dir}')

/content/instant-ngp/scripts
/content
Done with processing suites_bedroom to /content/drive/My Drive/cs231n_nerf/datasets/iphone_processed/original_processed/suites_bedroom


## Generating data splits (train, val, test)

In [ ]:
# creating train/val/test split for ALL PHOTOS in HashNeRF-pytorch baseline
import json, random, shutil
from pathlib import Path

# CHANGE NAME OF FOLDER FOR YOUR USE
room_name = 'suites_bedroom'

src = ORIG_ROOT / room_name
dst = ALLSHOT_ROOT / room_name
shutil.copytree(src, dst, dirs_exist_ok=True)

transforms_file = dst / 'transforms.json'

data = json.loads(transforms_file.read_text())
for frame in data['frames']:
  stem = Path(frame['file_path']).stem
  frame['file_path'] = f'images/{stem}'
transforms_file.write_text(json.dumps(data, indent=2))

frames = data['frames']
N = len(frames)
idxs = list(range(N))
random.seed(0)
random.shuffle(idxs)

n_train = int(0.8 * N)
n_val = int(0.1 * N)
train_idxs = idxs[:n_train]
val_idxs = idxs[n_train : n_train + n_val]
test_idxs = idxs[n_train + n_val : ]

for split, sel in [('train', train_idxs), ('val', val_idxs), ('test', test_idxs)]:
  split_data = dict(data)
  split_data['frames'] = [frames[i] for i in sel]
  out_path = dst / f"transforms_{split}.json"
  out_path.write_text(json.dumps(split_data, indent=2))

In [ ]:
# creating train/val/test split for FEW-SHOT approach in HashNeRF-pytorch baseline
import json, random, shutil
from pathlib import Path

# CHANGE NAME OF FOLDER FOR YOUR USE
room_name = 'suites_bedroom'

src = ORIG_ROOT / room_name
dst = FEWSHOT_ROOT / room_name
shutil.copytree(src, dst, dirs_exist_ok=True)

transforms_file = dst / 'transforms.json'
data = json.loads(transforms_file.read_text())
for frame in data['frames']:
  stem = Path(frame['file_path']).stem
  frame['file_path'] = f'images/{stem}'
transforms_file.write_text(json.dumps(data, indent=2))

frames = data['frames']
N = len(frames)
idxs = list(range(N))
random.seed(0)
random.shuffle(idxs)

n_train = 8
n_val = 2
n_test = 20

train_idxs = sorted(idxs[:n_train])
val_idxs = sorted(idxs[n_train : n_train + n_val])
test_idxs = sorted(idxs[n_train + n_val : n_train + n_val + n_test])

for split, sel in [('train', train_idxs), ('val', val_idxs), ('test', test_idxs)]:
  split_data = dict(data)
  split_data['frames'] = [frames[i] for i in sel]
  out_path = dst / f"transforms_{split}.json"
  out_path.write_text(json.dumps(split_data, indent=2))

## Transferring Data from Google Drive to VM Instance

In [ ]:
# START OF DATA TRANSFER FROM GOOGLE DRIVE TO VM INSTANCE
!gcloud auth login --no-launch-browser

In [ ]:
# ONLY RUN IF YOU WANT TO REPLACE EVERYTHING IN GCS BUCKET
!gsutil -m rm -r "gs://cs231n-data-459006/iphone_processed/*"

Removing gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room/colmap.db#1747483525958310...
Removing gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room/images/00000000.png#1747483536591898...
Removing gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room/images/00000001.png#1747483538245042...
Removing gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room/images/00000002.png#1747483529803518...
Removing gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room/images/00000004.png#1747483539097462...
Removing gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room/images/00000005.png#1747483538502918...
Removing gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room/images/00000003.png#1747483534904478...
Removing gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room/images/00000006.png#1747483530949668...
Removing gs://cs231n-data-459006/iphone_processed/

In [ ]:
# sending all room folder files in all shot & few shot folders to GCS bucket
from google.colab import auth
from pathlib import Path
import subprocess

auth.authenticate_user()

PROJECT_ID = 'cs231n-project-459006'
BUCKET = 'gs://cs231n-data-459006'

DST_ROOT = f'{BUCKET}/iphone_processed'
DST_ALL = f'{DST_ROOT}/allshot'
DST_FEW = f'{DST_ROOT}/fewshot'

OUT_ROOT = Path('/content/drive/My Drive/cs231n_nerf/datasets/iphone_processed')

for mode, subdir in [
    ('allshot_hashnerf_processed', 'allshot'),
    ('fewshot_hashnerf_processed', 'fewshot')
]:
  local = OUT_ROOT / mode
  dst = f'{DST_ROOT}/{subdir}'
  if not local.is_dir():
    print(f'Missing folder {local}')
    continue

  for room_dir in local.iterdir():
    if not room_dir.is_dir():
      print(f'Missing folder {room_dir.name}')
      continue

    print(f'Uploading {room_dir.name} to {dst}/{room_dir.name}')
    subprocess.run(['gsutil', '-m', 'cp', '-r', str(room_dir), f'{dst}/{room_dir.name}'], check=True)

Uploading norcliffe_common_room to gs://cs231n-data-459006/iphone_processed/allshot/norcliffe_common_room
Uploading suites_bedroom to gs://cs231n-data-459006/iphone_processed/allshot/suites_bedroom
Uploading norcliffe_common_room to gs://cs231n-data-459006/iphone_processed/fewshot/norcliffe_common_room
Uploading suites_bedroom to gs://cs231n-data-459006/iphone_processed/fewshot/suites_bedroom


In [ ]:
!gcloud config set project cs231n-project-459006
!gcloud config set compute/zone asia-northeast3-c

!VM_NAME=nvidia-nerf-vm
!REMOTE_FILE='~/norcliffe_baseline_hashXYZ_sphereVIEW_fine1024_log2T19_lr0.0005_decay500_RAdam_sparse1e-10_TV1e-06_spiral_005000_disp.mp4'

!gcloud compute scp ${VM_NAME}:${REMOTE_FILE}  ./

!open *_disp.mp4

Updated property [core/project].
ERROR: (gcloud.config.set) You do not currently have an active account selected.
Please run:

  $ gcloud auth login

to obtain new credentials.

If you have already logged in with a different account, run:

  $ gcloud config set account ACCOUNT

to select an already authenticated account to use.
This tool needs to create the directory [/root/.ssh] before being able to 
generate SSH keys.

Do you want to continue (Y/n)?  

Command killed by keyboard interrupt

^C
xdg-open: file '*_disp.mp4' does not exist
